In [1]:
class Property(object):
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [2]:
class C(object):
    def __init__(self):
        self.__x = None
    def getx(self): return self.__x
    def setx(self, value): self.__x = value
    def delx(self): del self.__x
    x = Property(getx, setx, delx, "I'm the 'x' property.")

描述器可以直接这么调用： d.__get__(obj)
然而更常见的情况是描述器在属性访问时被自动调用。举例来说， obj.d 会在 obj 的字典中找 d ,
如果 d 定义了 __get__ 方法，那么 d.__get__(obj) 会依据下面的优先规则被调用。
调用的细节取决于 obj 是一个类还是一个实例。另外，描述器只对于新式对象和新式类才起作用。继承于 object 的类叫做新式类。
对于对象来讲，方法 object.__getattribute__() 把 b.x 变成 type(b).__dict__['x'].__get__(b, type(b)) (x 自身当作self传入)。
具体实现是依据这样的优先顺序：资料描述器优先于实例变量，实例变量优先于非资料描述器，__getattr__()方法(如果对象中包含的话)具有最低的优先级。
完整的C语言实现可以在 Objects/object.c 中 PyObject_GenericGetAttr() 查看。

In [3]:
c = C()
print(c.x)
c.x = 1
print(c.x)

None
1


如果描述符属性名和设置属性同名，会导致嵌套递归：  
self.x = num 触发描述符协议，导致 x.__get__() 被调用，从而被注册的 fget 函数调用（self 参数为类 D 的实例），  
而该函数实体 self.x = num，将再次导致 触发描述符协议，从而无穷递归。

In [4]:
class D:
    def __init__(self, num):
        self.x = num
    @Property
    def x(self):
        return self.x
    @x.setter
    def x(self, num):
        self.x = num
try:
    d = D(1)
except RecursionError as e:
    print(repr(e))

RecursionError('maximum recursion depth exceeded while calling a Python object',)


class D:
    def __init__(self, num):
        self._x = num
    @property
    def x(self):
        return self._x
    @x.setter
    def x(self, num):
        self._x = num
    

修改设置的属性名称可以消除递归调用：

In [5]:
class D:
    def __init__(self, num):
        self._x = num
    @Property
    def x(self):
        return self._x
    @x.setter
    def x(self, num):
        if num < 0 :
            raise ValueError("must greater than 0")
        self._x = num

d = D(1)
print(d.x)
d.x = 2
print(d.x)
try:
    d.x = -5
except ValueError as e:
    print(e)

1
2
must greater than 0


但是，不能防止用户直接访问 _x 属性：

In [6]:
print(vars(d))
d._x = -5
print(d.x)
print(vars(d))

{'_x': 2}
-5
{'_x': -5}


## inspired by



## copyright

author：[bigfish](https://github.com/TheBigFish)  
copyright: [许可协议 知识共享署名-非商业性使用 4.0 国际许可协议](https://creativecommons.org/licenses/by-nc/4.0/)